In [12]:
import pandas as pd
import folium
from folium.plugins import MiniMap
from pathlib import Path
import json

# === 1. Baca data dari Google Sheets ===
url = "https://docs.google.com/spreadsheets/d/1iNevuUEhHf16MPlA-WDD1vDxiTRc5A4hGeBHZjh4XPk/export?format=csv"
data = pd.read_csv(url)

data.columns = data.columns.str.strip().str.upper()
data["JENIS"] = data["JENIS"].astype(str).str.strip().fillna("Lainnya")

# === 2. Lokasi awal peta ===
center_lat = data["LINTANG"].mean()
center_lon = data["BUJUR"].mean()

# === 3. Peta utama ===
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=6,
    tiles="OpenStreetMap",
    zoom_control=False
)

# Basemap tambahan
folium.TileLayer("cartodb positron", name="Carto Light").add_to(m)
folium.TileLayer("cartodb dark_matter", name="Carto Dark").add_to(m)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles © Esri",
    name="Esri Satellite"
).add_to(m)

# === 4. Layer Kawasan Konservasi ===
geojson_path = Path("Kawasan-Konservasi.json")
with open(geojson_path, encoding="utf-8") as f:
    kawasan_data = json.load(f)

folium.GeoJson(
    kawasan_data,
    name="Kawasan Konservasi",
    style_function=lambda feature: {
        'fillColor': '#FFFF00',
        'color': '#FFFF00',
        'weight': 0,
        'fillOpacity': 0.5,
    }
).add_to(m)

# === 5. Warna default ===
warna_dict = {
    "Penyu": "green",
    "Duyung": "blue",
    "Kima": "purple",
    "Lumba-lumba": "orange",
    "Lainnya": "red"
}

# === 6. Checkbox "Centang Semua" ===
checkall_js = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var interval = setInterval(function() {
        var control = document.querySelector('.leaflet-control-layers-overlays');
        if (control) {
            clearInterval(interval);
            var wrapper = document.createElement('div');
            wrapper.innerHTML = `
                <label style="font-weight:600; display:flex; align-items:center; margin-bottom:4px;">
                    <input type="checkbox" id="toggle-all" style="margin-right:6px;">
                    Filter / Hapus Filter
                </label>
            `;
            control.insertBefore(wrapper, control.firstChild);
            var toggleAll = document.getElementById('toggle-all');
            toggleAll.addEventListener('change', function() {
                var boxes = control.querySelectorAll('input.leaflet-control-layers-selector[type="checkbox"]');
                boxes.forEach(function(box) {
                    if (box.checked !== toggleAll.checked) box.click();
                });
            });
        }
    }, 300);
});
</script>
"""
m.get_root().html.add_child(folium.Element(checkall_js))

# === 7. Marker berdasarkan jenis ===
base_icon_url = "https://bkkpn-kupang.github.io/Imageicon/"

for jenis in data["JENIS"].unique():
    group = folium.FeatureGroup(name=f"{jenis}", show=True).add_to(m)
    subset = data[data["JENIS"] == jenis]

    for _, row in subset.iterrows():
        popup_text = f"""
        <b>KELOMPOK:</b> {row.get('KELOMPOK','')}<br>
        <b>SPESIES:</b> {row.get('SPESIES','')}<br>
        <b>NAMA ID:</b> {row.get('NAMA_ID','')}<br>
        <b>JUMLAH TEMU:</b> {row.get('JUMLAH_TEMU','')}<br>
        <b>DESKRIPSI:</b> {row.get('DESKRIPSI','')}<br>
        <b>TAHUN:</b> {row.get('TAHUN','')}<br>
        <b>LINK LAPOR:</b> <a href="{row.get('LINK_LAPOR','#')}" target="_blank">Klik di sini</a>
        """
        icon_filename = f"icon{jenis.lower().replace(' ', '').replace('-', '')}.png"
        icon_url = f"{base_icon_url}{icon_filename}"
        icon = folium.CustomIcon(icon_image=icon_url, icon_size=(40, 40))

        folium.Marker(
            location=[row["LINTANG"], row["BUJUR"]],
            popup=folium.Popup(popup_text, max_width=450),
            tooltip=row.get("SPESIES", ""),
            icon=icon
        ).add_to(group)

# === 8. LayerControl ===
folium.LayerControl(collapsed=False).add_to(m)

# === 9. Judul & tombol responsif ===
title_html = """
<div class="info-box">
    <div class="logo-container">
        <img src="https://bkkpn-kupang.github.io/Logo/LogoKKP.png" alt="Logo KKP" class="logo-item">
        <img src="https://bkkpn-kupang.github.io/Logo/LogoBKKPN.png" alt="Logo BKKPN Kupang" class="logo-item">
    </div>

    <div class="title-text">🌊 PETA SEBARAN BIOTA LAUT DILINDUNGI</div>
    <div class="subtitle-text">Balai Kawasan Konservasi Perairan Nasional Kupang</div>

    <div class="desc-text">Klik tombol di bawah untuk melihat grafik monitoring dan dashboard:</div>

    <div class="button-container">
        <a href="https://lookerstudio.google.com/u/0/reporting/faefd501-627c-47bf-b741-b25ebb73af8c/page/p_tu61f29uwd"
           target="_blank" class="btn blue">📊 Lihat Grafik Monitoring</a>
        <a href="https://sites.google.com/view/biota-laut-dilindungi/home"
           target="_blank" class="btn green">🌐 Buka Dashboard SI MOLI</a>
    </div>
</div>

<style>
.info-box {
    position: fixed;
    top: 15px;
    left: 20px;
    background-color: rgba(255, 255, 255, 0.95);
    padding: 24px 28px;
    border-radius: 14px;
    font-family: 'Segoe UI', Arial, sans-serif;
    box-shadow: 0 4px 12px rgba(0,0,0,0.25);
    z-index: 9999;
    max-width: 380px;
    text-align: center;
    backdrop-filter: blur(6px);
}

.logo-container {
    display: flex;
    justify-content: center;
    align-items: center;
    gap: 18px;
    margin-bottom: 12px;
}

.logo-item {
    height: 42px;
    object-fit: contain;
}

.title-text {
    font-size: 22px;
    font-weight: 800;
    color: #0b3d91;
    margin-bottom: 8px;
}

.subtitle-text {
    font-size: 14px;
    font-weight: 600;
    color: #004080;
    margin-bottom: 10px;
}

.desc-text {
    font-size: 15px;
    color: #2c3e50;
    line-height: 1.4;
}

.button-container {
    margin-top: 10px;
    display: flex;
    flex-direction: column;
    gap: 8px;
}

.btn {
    display: block;
    color: white;
    text-decoration: none;
    padding: 9px;
    border-radius: 8px;
    font-weight: 600;
    transition: 0.3s;
    font-size: 15px;
}

.btn.blue { background-color: #007BFF; }
.btn.green { background-color: #28a745; }
.btn.blue:hover { background-color: #0056b3; }
.btn.green:hover { background-color: #1e7e34; }

/* === Responsif untuk tablet & HP === */
@media (max-width: 768px) {
    .info-box {
        max-width: 50%;
        left: 25%;
        transform: translateX(-50%);
        top: 10px;
        padding: 16px;
    }
    .logo-container {
        flex-direction: column;
        gap: 6px;
    }
    .logo-item {
        height: 36px;
    }
    .title-text {
        font-size: 18px;
    }
    .subtitle-text {
        font-size: 12px;
    }
    .desc-text {
        font-size: 13px;
    }
    .btn {
        font-size: 13px;
        padding: 8px;
    }
    .leaflet-top.leaflet-right {
        top: 120px !important;
        right: 10px !important;
    }
    .leaflet-popup-content {
        max-width: 230px !important;
        font-size: 12px;
    }
}
</style>
"""
m.get_root().html.add_child(folium.Element(title_html))



# === 9. Tambahkan Legenda Otomatis (pakai ikon dari GitHub) - VERSI FIX RESPONSIF ===
legend_html = f"""
<!-- Legend container -->
<div id="legend-wrapper" style="position: fixed; z-index: 99999; bottom: 20px; left: 20px;">
  <div id="legend" class="legend-open" style="
       background: rgba(255,255,255,0.96);
       padding: 10px 12px;
       border-radius: 10px;
       box-shadow: 0 3px 10px rgba(0,0,0,0.18);
       font-family: 'Segoe UI', Arial, sans-serif;
       font-size: 14px;
       line-height: 1.4;
       min-width: 160px;
       max-width: 260px;
  ">
    <div style="display:flex; align-items:center; justify-content:space-between; gap:8px;">
      <b style="color:#0b3d91; font-size:15px;">🗺️ Legenda Biota</b>
      <button id="legend-toggle" aria-expanded="true" style="
              background:transparent; border:0; cursor:pointer; font-size:16px; line-height:1;
              color:#0b3d91; padding:2px 6px;">✕
      </button>
    </div>
    <div id="legend-items" style="margin-top:8px;">
"""

# build legend items (icons + label)
for jenis in data["JENIS"].unique():
    safe_name = str(jenis)
    icon_filename = f"icon{safe_name.lower().replace(' ', '').replace('-', '')}.png"
    icon_url = f"{base_icon_url}{icon_filename}"
    # item HTML: use onerror to fallback to a small colored box if image missing
    legend_html += f"""
      <div style="display:flex; align-items:center; gap:8px; margin-top:6px;">
        <img src="{icon_url}" width="28" height="28" style="border-radius:4px; object-fit:contain;"
             onerror="this.style.display='none'; this.nextElementSibling.style.display='inline-block';">
        <span style="display:none; width:28px; height:28px; border-radius:4px; background:#ddd;"></span>
        <span style="white-space:nowrap; overflow:hidden; text-overflow:ellipsis; display:inline-block; max-width:150px;">{safe_name}</span>
      </div>
    """

legend_html += """
    </div>
  </div>
</div>

<!-- Legend CSS responsif + simple JS toggle -->
<style>
  /* small screens: make legend compact and closable */
  @media (max-width: 768px) {
    #legend {
      min-width: 140px;
      max-width: 85vw;
      padding: 8px 10px;
      font-size: 13px;
    }
    #legend-items { display: none; } /* start collapsed on small screens */
    #legend.legend-open #legend-items { display: block; }
    /* move layer control a bit down so they don't overlap */
    .leaflet-top.leaflet-right { top: 120px !important; }
  }
  /* pointer improvements */
  #legend button { background: transparent; border: none; cursor: pointer; }
</style>

<script>
  (function() {
    // Toggle behaviour for legend (keeps legend closed on small screens initially)
    var wrapper = document.getElementById('legend-wrapper');
    var legend = wrapper && wrapper.querySelector('#legend');
    var toggle = wrapper && wrapper.querySelector('#legend-toggle');
    var items = wrapper && wrapper.querySelector('#legend-items');

    if (!legend || !toggle || !items) return;

    // if screen small, collapse by default
    function setInitialState() {
      if (window.innerWidth <= 768) {
        legend.classList.remove('legend-open');
        toggle.textContent = '☰'; // menu icon
        toggle.setAttribute('aria-expanded', 'false');
      } else {
        legend.classList.add('legend-open');
        toggle.textContent = '✕';
        toggle.setAttribute('aria-expanded', 'true');
      }
    }

    setInitialState();

    // toggle click
    toggle.addEventListener('click', function(e) {
      if (legend.classList.contains('legend-open')) {
        legend.classList.remove('legend-open');
        toggle.textContent = '☰';
        toggle.setAttribute('aria-expanded', 'false');
      } else {
        legend.classList.add('legend-open');
        toggle.textContent = '✕';
        toggle.setAttribute('aria-expanded', 'true');
      }
    });

    // also update on resize
    window.addEventListener('resize', setInitialState);
  })();
</script>
"""
# inject into folium
m.get_root().html.add_child(folium.Element(legend_html))

# === 10. Simpan file HTML ===
Path("index.html").write_text(m.get_root().render(), encoding="utf-8")
print("✅ Peta berhasil dibuat → buka file: index.html")


✅ Peta berhasil dibuat → buka file: index.html
